# Song Generator

Idea: use the chord progressions thing to generate sequence, then a melody over that sequence that fits according to the number of beats in an input poem

![](poss_chords_prog.png)

---

### Structure
1. Generate chord sequence
2. Do this for verses, chorus, and bridge
3. Parse input test, find number of syllables per line
4. Generate melody with same number of notes over the chord progression
5. (maybe) export to midi files
6. Put into ableton
7. Record vocals
8. Repeat over and over and over...
9. Profit

In [138]:
import random
from midiutil.MidiFile import MIDIFile
from datetime import datetime

In [260]:
class SongGeneration:
    
    def __init__(self, key, length=4, rhythm=[]):
        self.key = key
        self.length = length # default is 4 chords
        # absolute maximum recommended length: 9
        
        # Chord Progression Dictionary
        self.chord_prog_maj = {'I':[['V', 'vii'], ['ii', 'IV'], ['vi'], ['iii']],
                          'ii':['V', 'vii'],
                          'iii': ['vi'],
                          'IV':['V', 'vii'],
                          'V': ['iii'],
                          'vi': ['ii', 'IV'],
                          'vii': ['iii']}
        
        # List of all possible notes and rhythms, plus the weights for picking the rhythm
        self.all_notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.all_rhythms = [0.25, 0.5, 1, 2]
        self.all_rhythm_weights = [0.1, 0.4, 0.95, 1.0]
        
        # Notes dictionary for MIDI file
        self.notes_dict = {}
        bass = 48
        treble1 = 60
        treble2 = 72
        for i in range(0, len(self.all_notes)):
            self.notes_dict[self.all_notes[i]] = [bass + i, treble1 + i, treble2 + i]
        
        # Class variables, for use in generation functions
        self.scale = []
        self.chords = []
        self.chord_notes = []
        self.melody = []
        self.rhythm = rhythm
    
    def __str__(self):
        return_string = str(self.scale) + "\n" + str(self.chords) + "\n" 
        return_string += str(self.chord_notes) + "\n" + str(self.rhythm) + "\n"
        return_string += str(self.melody)
        return return_string
    
    def gen_song(self):
        self.gen_scale()
        self.gen_chords()
        self.gen_rhythm()
        self.gen_melody()
        self.gen_MIDI()
        
    def gen_scale(self):
        i = self.all_notes.index(self.key)
        l = len(self.all_notes)
        self.scale.append(self.all_notes[i])
        self.scale.append(self.all_notes[(i + 2) % l])
        self.scale.append(self.all_notes[(i + 4) % l])
        self.scale.append(self.all_notes[(i + 5) % l])
        self.scale.append(self.all_notes[(i + 7) % l])
        self.scale.append(self.all_notes[(i + 9) % l])
        self.scale.append(self.all_notes[(i + 11) % l])
        self.scale.append(self.all_notes[i])
        
    def gen_chords(self):
        chords = ['I']
        sub_value = 2
        start_value = (self.length - sub_value) % 4
        start_chord = random.choice(self.chord_prog_maj[chords[0]][start_value])
        chords.append(start_chord)
        
        while len(chords) != self.length:
            next_chord = random.choice(self.chord_prog_maj[chords[-1]])            
            chords.append(next_chord)
            
        self.chords = chords
        
        for chord in self.chords:
            if chord == 'I':
                self.chord_notes.append([self.scale[0], self.scale[2], self.scale[4]])
            elif chord == 'ii':
                self.chord_notes.append([self.scale[1], self.scale[3], self.scale[5]])
            elif chord == 'iii':
                self.chord_notes.append([self.scale[2], self.scale[4], self.scale[6]])
            elif chord == 'IV':
                self.chord_notes.append([self.scale[3], self.scale[5], self.scale[0]])
            elif chord == 'V':
                self.chord_notes.append([self.scale[4], self.scale[6], self.scale[1]])
            elif chord == 'vi':
                self.chord_notes.append([self.scale[5], self.scale[0], self.scale[2]])
            elif chord == 'vii':
                self.chord_notes.append([self.scale[6], self.scale[1], self.scale[3]])
        #print(self.chords)
    
    def gen_rhythm(self):
        if len(self.rhythm) == 0:
            for chord in self.chords:
                total = 0
                measure = []
                while total < 4:
                    r = random.uniform(0, 1)
                    #new_note = random.choice(self.all_rhythms)
                    for i in range(0, len(self.all_rhythm_weights)):
                        if self.all_rhythm_weights[i] <= r:
                            new_note = self.all_rhythms[i]
                    if total + new_note > 4:
                        new_note = 4 - total
                    measure.append(new_note)
                    total += new_note
                self.rhythm.append(measure)
                
    
    def gen_melody(self):
        for i in range(0, len(self.chords)):
            starting_note = random.choice(self.chord_notes[i]) 
            measure = [[starting_note, self.rhythm[i][0]]]
            for j in range(1, len(self.rhythm[i])):
                prev_note = measure[j-1][0]
                scale_i = self.scale.index(prev_note)
                if self.rhythm[i][j] < 2:
                    note = random.choice([self.scale[(scale_i + len(self.scale) - 1) % len(self.scale)], 
                                          self.scale[(scale_i + len(self.scale) + 1) % len(self.scale)]])
                else:
                    note = random.choice(self.chord_notes[i]) 
                measure.append([note, self.rhythm[i][j]])
            self.melody.append(measure)
    
    def gen_MIDI(self):
        #print(str(self.melody))
        # create your MIDI object
        mf = MIDIFile(1)     # only 1 track
        track = 0   # the only track

        time = 0    # start at the beginning
        mf.addTrackName(track, time, "Test")
        mf.addTempo(track, time, 120)

        # add some notes
        channel = 0
        volume = 100
        
        for i in range(0, len(self.chord_notes)):
            chord = self.chord_notes[i]
            chord_time = i * 4
            pitch1 = self.notes_dict[chord[0]][0]
            pitch2 = self.notes_dict[chord[1]][0]
            pitch3 = self.notes_dict[chord[2]][0]
            duration = 4
            mf.addNote(track, channel, pitch1, chord_time, duration, 65)
            mf.addNote(track, channel, pitch2, chord_time, duration, 65)
            mf.addNote(track, channel, pitch3, chord_time, duration, 65)
        
        last_pitch = 0
        for i in range(0, len(self.melody)):
            measure = self.melody[i]
            measure_time = i * 4
            
            for j in range(0, len(measure)):
                note = measure[j][0]
                pitch1 = self.notes_dict[note][1]
                pitch2 = self.notes_dict[note][2]
                
                dist1 = (pitch1 - last_pitch)**2
                dist2 = (pitch2 - last_pitch)**2
                
                pitch = pitch1 if dist1 <= dist2 else pitch2                
                duration = measure[j][1]
                mf.addNote(track, channel, pitch, measure_time, duration, volume)
                
                measure_time += measure[j][1]
                last_pitch = pitch
            

        # write it to disk
        now = str(datetime.now()) # current date and time
        now = now.replace(":", "")
        now = now.replace(" ", "_")
        now = now.replace(".", "_")
        with open("midi_files\\" + "test" + ".mid", 'wb') as outf:
            mf.writeFile(outf)

In [262]:
test_song = SongGeneration('C', length=4)
test_song.gen_song()
print(test_song)

['C', 'D', 'E', 'F', 'G', 'A', 'B', 'C']
['I', 'vi', 'IV', 'V']
[['C', 'E', 'G'], ['A', 'C', 'E'], ['F', 'A', 'C'], ['G', 'B', 'D']]
[[0.5, 0.5, 0.5, 1, 0.5, 0.5, 0.5], [0.5, 0.25, 1, 0.5, 0.25, 0.5, 0.5, 0.5], [0.5, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.25, 0.5, 0.5], [1, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.25, 0.5, 0.5, 0.25], [0.25, 0.25, 0.25, 0.25, 0.25, 1, 0.25, 0.5, 0.25, 0.5, 0.25], [0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 1, 0.5]]
[[['G', 0.5], ['F', 0.5], ['G', 0.5], ['F', 1], ['E', 0.5], ['D', 0.5], ['E', 0.5]], [['E', 0.5], ['F', 0.25], ['G', 1], ['A', 0.5], ['G', 0.25], ['A', 0.5], ['B', 0.5], ['A', 0.5]], [['C', 0.5], ['C', 0.5], ['D', 0.25], ['C', 0.25], ['C', 0.25], ['D', 0.25], ['C', 0.25], ['D', 0.5], ['E', 0.25], ['F', 0.5], ['G', 0.5]], [['D', 1], ['E', 0.5], ['F', 0.5], ['E', 0.5], ['F', 0.5], ['G', 0.25], ['A', 0.25], ['G